In [101]:
# Using beautiful soup to scrape ranchi blocks
from bs4 import BeautifulSoup
import requests

In [131]:
def get_soup_object(url):
    source_data = requests.get(url).text
    return BeautifulSoup(source_data,'lxml')

In [171]:
# initialize url
rnc_data_url = 'http://vlist.in/district/364.html'
# use function to get soup object
soup = get_soup_object(rnc_data_url)
print('Soup object created')

Soup object created


In [172]:
village_url_header = 'http://vlist.in'
district_name = 'Ranchi'

In [173]:
# function extracts row from the table from government website. This will return the name in the table and the link associated with the name
def extract_row(table_row):
    table_row = table_row.find_all('td')
    
    index = table_row[0].text
    
    link = village_url_header + table_row[1].find('a')['href']
    
    name = table_row[1].text
    
    return link, name

In [206]:
# extracting the block rows
table_rows = soup.find_all('tr')
table_rows = table_rows[1:]
table_rows = table_rows[1:]
data = []
# for every block row all the villages will also be extracted
for table_row in table_rows:
    
    sub_district_link, block_name = extract_row(table_row)
    print(block_name)
    # getting the sub villages in block
    soup_village = get_soup_object(sub_district_link)
    # get all the table rows for individual villages in block
    sub_table_rows = soup_village.find_all('tr')
    sub_table_rows = sub_table_rows[1:]
    
    # extract individual village name and store it in data along with block name and district name
    for sub_table_row in sub_table_rows:
    
        sub_link, village_name = extract_row(sub_table_row)
        
        data.append([village_name, block_name, district_name])

print(data)

Namkum
Ormanjhi
Rahe
Ratu
Silli
Sonahatu
Tamar I
[['Ara (CT)', 'Namkum', 'Ranchi'], ['Arma', 'Namkum', 'Ranchi'], ['Banda', 'Namkum', 'Ranchi'], ['Bandua', 'Namkum', 'Ranchi'], ['Baram', 'Namkum', 'Ranchi'], ['Bargarwa (CT)', 'Namkum', 'Ranchi'], ['Barguttu', 'Namkum', 'Ranchi'], ['Bumru', 'Namkum', 'Ranchi'], ['Bundubera', 'Namkum', 'Ranchi'], ['Butiyo', 'Namkum', 'Ranchi'], ['Chand', 'Namkum', 'Ranchi'], ['Chandaghasi', 'Namkum', 'Ranchi'], ['Chene', 'Namkum', 'Ranchi'], ['Chitir', 'Namkum', 'Ranchi'], ['Churu', 'Namkum', 'Ranchi'], ['Darjitorar', 'Namkum', 'Ranchi'], ['Dewgain', 'Namkum', 'Ranchi'], ['Dundigarha', 'Namkum', 'Ranchi'], ['Dundu', 'Namkum', 'Ranchi'], ['Dungri', 'Namkum', 'Ranchi'], ['Ganrke', 'Namkum', 'Ranchi'], ['Garhkhatanga', 'Namkum', 'Ranchi'], ['Garsul', 'Namkum', 'Ranchi'], ['Ghuthiya', 'Namkum', 'Ranchi'], ['Gobindpur', 'Namkum', 'Ranchi'], ['Gundu', 'Namkum', 'Ranchi'], ['Guniyadih', 'Namkum', 'Ranchi'], ['Hahap', 'Namkum', 'Ranchi'], ['Hajam', 'Namkum', 'Ra

In [207]:
# save data in csv for future usage
header = ['Village','Block','District']
df = pd.DataFrame(data= data, columns= header)

In [208]:
df.head()

,Village,Block,District
0,Ara (CT),Namkum,Ranchi
1,Arma,Namkum,Ranchi
2,Banda,Namkum,Ranchi
3,Bandua,Namkum,Ranchi
4,Baram,Namkum,Ranchi


In [213]:
df['Block'].unique()

array(['Namkum', 'Ormanjhi', 'Rahe', 'Ratu', 'Silli', 'Sonahatu',
       'Tamar I'], dtype=object)

In [219]:
df = pd.read_csv('ranchi_villages.csv')

,Unnamed: 0,Village,Block,District
1899,573,Timpur,Tamar I,Ranchi
1900,574,Toranga,Tamar I,Ranchi
1901,575,Udaidih,Tamar I,Ranchi
1902,576,Ulidih,Tamar I,Ranchi
1903,577,Ulilohar,Tamar I,Ranchi


In [222]:
import pandas as pd
import numpy as np

In [223]:
# use geocoder library, if not present use !conda install -c conda-forge geocoder
import geocoder
# Google API key is required for the geocoder library to work, save the API key in OS environment variables as GOOGLE_API_KEY
# and then access thay key here
import os
# Use BING_API_KEY when choosing to use bing geocoding instead of google geocoding.
BING_API_KEY = 'AksNN-3luSfNBssyZ3Ju4i78nIrFLt1UtYo--YWQj9oyfxSwyXkdsqykWk3FeTXB' # os.environ['BING_API_KEY']

In [224]:
# This function will take an adress and return the latlng of that adress
def get_latlng(address):
    # using bing geocoder API since it is better.
    g = geocoder.bing(address, key = BING_API_KEY)
    return pd.Series(g.latlng)

In [225]:
# using the get_latlng function to define latitude and longitude columns of the data frame
df[['Latitude','Longitude']] = df.apply(lambda x: get_latlng(x.Village +', '+ x.Block + ', ' + x.District), axis=1)
df.head()

Status code Unknown from http://dev.virtualearth.net/REST/v1/Locations: ERROR - HTTPConnectionPool(host='dev.virtualearth.net', port=80): Read timed out. (read timeout=5.0)
Status code 429 from http://dev.virtualearth.net/REST/v1/Locations: ERROR - 429 Client Error: Too Many Requests for url: http://dev.virtualearth.net/REST/v1/Locations?q=Patagain%2C+Kanke%2C+Ranchi&o=json&inclnb=1&key=AksNN-3luSfNBssyZ3Ju4i78nIrFLt1UtYo--YWQj9oyfxSwyXkdsqykWk3FeTXB&maxResults=1
Status code Unknown from http://dev.virtualearth.net/REST/v1/Locations: ERROR - HTTPConnectionPool(host='dev.virtualearth.net', port=80): Read timed out. (read timeout=5.0)
Status code Unknown from http://dev.virtualearth.net/REST/v1/Locations: ERROR - HTTPConnectionPool(host='dev.virtualearth.net', port=80): Read timed out. (read timeout=5.0)
Status code Unknown from http://dev.virtualearth.net/REST/v1/Locations: ERROR - HTTPConnectionPool(host='dev.virtualearth.net', port=80): Read timed out. (read timeout=5.0)
Status code U

Status code Unknown from http://dev.virtualearth.net/REST/v1/Locations: ERROR - HTTPConnectionPool(host='dev.virtualearth.net', port=80): Max retries exceeded with url: /REST/v1/Locations?q=Babaikundi%2C+Tamar+I%2C+Ranchi&o=json&inclnb=1&key=AksNN-3luSfNBssyZ3Ju4i78nIrFLt1UtYo--YWQj9oyfxSwyXkdsqykWk3FeTXB&maxResults=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000289CCF2C8D0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Status code Unknown from http://dev.virtualearth.net/REST/v1/Locations: ERROR - HTTPConnectionPool(host='dev.virtualearth.net', port=80): Max retries exceeded with url: /REST/v1/Locations?q=Baburamdih%2C+Tamar+I%2C+Ranchi&o=json&inclnb=1&key=AksNN-3luSfNBssyZ3Ju4i78nIrFLt1UtYo--YWQj9oyfxSwyXkdsqykWk3FeTXB&maxResults=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000289CD2E31D0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Status code Unknow

Status code Unknown from http://dev.virtualearth.net/REST/v1/Locations: ERROR - HTTPConnectionPool(host='dev.virtualearth.net', port=80): Max retries exceeded with url: /REST/v1/Locations?q=Burudih%2C+Tamar+I%2C+Ranchi&o=json&inclnb=1&key=AksNN-3luSfNBssyZ3Ju4i78nIrFLt1UtYo--YWQj9oyfxSwyXkdsqykWk3FeTXB&maxResults=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000289CD1C3DA0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Status code Unknown from http://dev.virtualearth.net/REST/v1/Locations: ERROR - HTTPConnectionPool(host='dev.virtualearth.net', port=80): Max retries exceeded with url: /REST/v1/Locations?q=Burudih%2C+Tamar+I%2C+Ranchi&o=json&inclnb=1&key=AksNN-3luSfNBssyZ3Ju4i78nIrFLt1UtYo--YWQj9oyfxSwyXkdsqykWk3FeTXB&maxResults=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000289CD1C3E48>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Status code Unknown from

Status code Unknown from http://dev.virtualearth.net/REST/v1/Locations: ERROR - HTTPConnectionPool(host='dev.virtualearth.net', port=80): Max retries exceeded with url: /REST/v1/Locations?q=Dimbujarda%2C+Tamar+I%2C+Ranchi&o=json&inclnb=1&key=AksNN-3luSfNBssyZ3Ju4i78nIrFLt1UtYo--YWQj9oyfxSwyXkdsqykWk3FeTXB&maxResults=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000289CD461278>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Status code Unknown from http://dev.virtualearth.net/REST/v1/Locations: ERROR - HTTPConnectionPool(host='dev.virtualearth.net', port=80): Max retries exceeded with url: /REST/v1/Locations?q=Dimra%2C+Tamar+I%2C+Ranchi&o=json&inclnb=1&key=AksNN-3luSfNBssyZ3Ju4i78nIrFLt1UtYo--YWQj9oyfxSwyXkdsqykWk3FeTXB&maxResults=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000289CD4611D0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Status code Unknown fro

Status code Unknown from http://dev.virtualearth.net/REST/v1/Locations: ERROR - HTTPConnectionPool(host='dev.virtualearth.net', port=80): Max retries exceeded with url: /REST/v1/Locations?q=Haranlohar%2C+Tamar+I%2C+Ranchi&o=json&inclnb=1&key=AksNN-3luSfNBssyZ3Ju4i78nIrFLt1UtYo--YWQj9oyfxSwyXkdsqykWk3FeTXB&maxResults=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000289CD4E3550>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Status code Unknown from http://dev.virtualearth.net/REST/v1/Locations: ERROR - HTTPConnectionPool(host='dev.virtualearth.net', port=80): Max retries exceeded with url: /REST/v1/Locations?q=Hurundih%2C+Tamar+I%2C+Ranchi&o=json&inclnb=1&key=AksNN-3luSfNBssyZ3Ju4i78nIrFLt1UtYo--YWQj9oyfxSwyXkdsqykWk3FeTXB&maxResults=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000289CD4E3400>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Status code Unknown 

...

Status code Unknown from http://dev.virtualearth.net/REST/v1/Locations: ERROR - HTTPConnectionPool(host='dev.virtualearth.net', port=80): Max retries exceeded with url: /REST/v1/Locations?q=Kheduadih+Alias+Dindlidih%2C+Tamar+I%2C+Ranchi&o=json&inclnb=1&key=AksNN-3luSfNBssyZ3Ju4i78nIrFLt1UtYo--YWQj9oyfxSwyXkdsqykWk3FeTXB&maxResults=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000289CD5F4EB8>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Status code Unknown from http://dev.virtualearth.net/REST/v1/Locations: ERROR - HTTPConnectionPool(host='dev.virtualearth.net', port=80): Max retries exceeded with url: /REST/v1/Locations?q=Kitadih%2C+Tamar+I%2C+Ranchi&o=json&inclnb=1&key=AksNN-3luSfNBssyZ3Ju4i78nIrFLt1UtYo--YWQj9oyfxSwyXkdsqykWk3FeTXB&maxResults=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000289CD5F4F60>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Status

Status code Unknown from http://dev.virtualearth.net/REST/v1/Locations: ERROR - HTTPConnectionPool(host='dev.virtualearth.net', port=80): Max retries exceeded with url: /REST/v1/Locations?q=Lungtu%2C+Tamar+I%2C+Ranchi&o=json&inclnb=1&key=AksNN-3luSfNBssyZ3Ju4i78nIrFLt1UtYo--YWQj9oyfxSwyXkdsqykWk3FeTXB&maxResults=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000289CD5E0470>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Status code Unknown from http://dev.virtualearth.net/REST/v1/Locations: ERROR - HTTPConnectionPool(host='dev.virtualearth.net', port=80): Max retries exceeded with url: /REST/v1/Locations?q=Madhudih%2C+Tamar+I%2C+Ranchi&o=json&inclnb=1&key=AksNN-3luSfNBssyZ3Ju4i78nIrFLt1UtYo--YWQj9oyfxSwyXkdsqykWk3FeTXB&maxResults=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000289CD5E0780>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Status code Unknown from

Status code Unknown from http://dev.virtualearth.net/REST/v1/Locations: ERROR - HTTPConnectionPool(host='dev.virtualearth.net', port=80): Max retries exceeded with url: /REST/v1/Locations?q=Peraidih%2C+Tamar+I%2C+Ranchi&o=json&inclnb=1&key=AksNN-3luSfNBssyZ3Ju4i78nIrFLt1UtYo--YWQj9oyfxSwyXkdsqykWk3FeTXB&maxResults=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000289CD5E0E80>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Status code Unknown from http://dev.virtualearth.net/REST/v1/Locations: ERROR - HTTPConnectionPool(host='dev.virtualearth.net', port=80): Max retries exceeded with url: /REST/v1/Locations?q=Peyakuli%2C+Tamar+I%2C+Ranchi&o=json&inclnb=1&key=AksNN-3luSfNBssyZ3Ju4i78nIrFLt1UtYo--YWQj9oyfxSwyXkdsqykWk3FeTXB&maxResults=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000289CD5E0F28>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Status code Unknown fr

Status code Unknown from http://dev.virtualearth.net/REST/v1/Locations: ERROR - HTTPConnectionPool(host='dev.virtualearth.net', port=80): Max retries exceeded with url: /REST/v1/Locations?q=Tarai%2C+Tamar+I%2C+Ranchi&o=json&inclnb=1&key=AksNN-3luSfNBssyZ3Ju4i78nIrFLt1UtYo--YWQj9oyfxSwyXkdsqykWk3FeTXB&maxResults=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000289CD5DD5C0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Status code Unknown from http://dev.virtualearth.net/REST/v1/Locations: ERROR - HTTPConnectionPool(host='dev.virtualearth.net', port=80): Max retries exceeded with url: /REST/v1/Locations?q=Timpur%2C+Tamar+I%2C+Ranchi&o=json&inclnb=1&key=AksNN-3luSfNBssyZ3Ju4i78nIrFLt1UtYo--YWQj9oyfxSwyXkdsqykWk3FeTXB&maxResults=1 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000289CD5DD630>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))
Status code Unknown from ht

,Village,Block,District,Latitude,Longitude
0,Agartoli,Angara,Ranchi,23.376190,85.439278
1,Angara,Angara,Ranchi,23.376190,85.439278
2,Asri,Angara,Ranchi,23.357210,85.666901
3,Badri,Angara,Ranchi,23.432671,85.633270
4,Baheya,Angara,Ranchi,23.370399,85.487930


In [227]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1904 entries, 0 to 1903
Data columns (total 5 columns):
Village      1904 non-null object
Block        1904 non-null object
District     1904 non-null object
Latitude     1177 non-null float64
Longitude    1177 non-null float64
dtypes: float64(2), object(3)
memory usage: 74.5+ KB


In [228]:
#!conda install -c conda-forge folium --yes # uncomment this line if folium is missing
import folium

In [229]:
rnc_latitude = 23.3441
rnc_longitude = 85.3096

In [231]:
# create map of Toronto using latitude and longitude values
map_ranchi = folium.Map(location=[rnc_latitude, rnc_longitude], zoom_start=10)

# data to be used for map
data = df.dropna()

# add markers to map
for lat, lng, village, block in zip(data['Latitude'], data['Longitude'], data['Village'], data['Block']):
    label = '{}, {}'.format(village, block)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ranchi)  
    
map_ranchi

In [46]:
# @hidden
# initializing foursquare API credentials
CLIENT_ID = 'JFIPVNQGSM3DJVNB4KPAFDWCS2AN5MLT0CUXMBDEEZ3TFTN1' # your Foursquare ID
CLIENT_SECRET = 'O5MIDR0SDHPOMB4VRJRVJY2TV0YHYIYRKUUANCQKKN3UYOF4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

In [60]:
# intialize a function to get all the venue in the neighbourhood
def getNearbyVenues(names, latitudes, longitudes, radius=22000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [61]:
# use above function to get venues nearby toronto
# function takes a lot of time
ranchi_venues = getNearbyVenues(names=data['BLOCK NAME'],
                                   latitudes=data['Latitude'],
                                   longitudes=data['Longitude']
                                  )

ANGARA
BERO
BURMU
CHANHO
ITKI
KANKE
KHELARI
LAPUNG
MANDAR
NAMKUM
ORMANJHI
RATU
SILLI
NAGRI
BUNDU
RAHE
SONAHATU
TAMAR


In [63]:
# see number of venues per neighbourhood
ranchi_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ANGARA,15,15,15,15,15,15
BERO,1,1,1,1,1,1
BUNDU,3,3,3,3,3,3
BURMU,3,3,3,3,3,3
CHANHO,4,4,4,4,4,4
ITKI,13,13,13,13,13,13
KANKE,13,13,13,13,13,13
KHELARI,5,5,5,5,5,5
LAPUNG,1,1,1,1,1,1


In [64]:
# one hot encoding
ranchi_onehot = pd.get_dummies(ranchi_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ranchi_onehot['Neighborhood'] = ranchi_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ranchi_onehot.columns[-1]] + list(ranchi_onehot.columns[:-1])
ranchi_onehot = ranchi_onehot[fixed_columns]

ranchi_onehot.head()

,Neighborhood,ATM,Beach,Bookstore,Café,Clothing Store,Coffee Shop,Comfort Food Restaurant,Department Store,Fast Food Restaurant,...,Indian Chinese Restaurant,Indian Restaurant,Light Rail Station,Multiplex,Pizza Place,Platform,Scenic Lookout,Shopping Mall,Stadium,Train Station
0,ANGARA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ANGARA,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,ANGARA,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,ANGARA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,ANGARA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [70]:
# group the data per neighborhood
ranchi_grouped = ranchi_onehot.groupby('Neighborhood').count().reset_index()
ranchi_grouped

,Neighborhood,ATM,Beach,Bookstore,Café,Clothing Store,Coffee Shop,Comfort Food Restaurant,Department Store,Fast Food Restaurant,...,Indian Chinese Restaurant,Indian Restaurant,Light Rail Station,Multiplex,Pizza Place,Platform,Scenic Lookout,Shopping Mall,Stadium,Train Station
0,ANGARA,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15
1,BERO,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,BUNDU,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
3,BURMU,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,CHANHO,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4
5,ITKI,13,13,13,13,13,13,13,13,13,...,13,13,13,13,13,13,13,13,13,13
6,KANKE,13,13,13,13,13,13,13,13,13,...,13,13,13,13,13,13,13,13,13,13
7,KHELARI,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
8,LAPUNG,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9,MANDAR,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [67]:
# First, let's write a function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [69]:
# Now let's create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ranchi_grouped['Neighborhood']

for ind in np.arange(ranchi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ranchi_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ANGARA,Shopping Mall,Coffee Shop,Indian Restaurant,Train Station,Hotel Bar,Café,Department Store,Fast Food Restaurant,Stadium,Hotel
1,BERO,ATM,History Museum,Beach,Bookstore,Café,Clothing Store,Coffee Shop,Comfort Food Restaurant,Department Store,Fast Food Restaurant
2,BUNDU,ATM,History Museum,Fast Food Restaurant,Beach,Bookstore,Café,Clothing Store,Coffee Shop,Comfort Food Restaurant,Department Store
3,BURMU,ATM,Platform,Furniture / Home Store,History Museum,Beach,Bookstore,Café,Clothing Store,Coffee Shop,Comfort Food Restaurant
4,CHANHO,ATM,Bookstore,Indian Chinese Restaurant,History Museum,Beach,Café,Clothing Store,Coffee Shop,Comfort Food Restaurant,Department Store
5,ITKI,Indian Restaurant,Shopping Mall,Multiplex,Train Station,Café,Fast Food Restaurant,Stadium,Hotel Bar,Hotel,Pizza Place
6,KANKE,Shopping Mall,Indian Restaurant,Coffee Shop,Hotel,Fast Food Restaurant,Pizza Place,Multiplex,Café,Hotel Bar,Department Store
7,KHELARI,ATM,Beach,Scenic Lookout,Comfort Food Restaurant,Indian Chinese Restaurant,Furniture / Home Store,Bookstore,Café,Clothing Store,Coffee Shop
8,LAPUNG,Clothing Store,Train Station,History Museum,Beach,Bookstore,Café,Coffee Shop,Comfort Food Restaurant,Department Store,Fast Food Restaurant
9,MANDAR,ATM,Furniture / Home Store,Train Station,Scenic Lookout,Platform,Beach,Bookstore,Café,Clothing Store,Coffee Shop
